In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from rich.jupyter import print

<IPython.core.display.Javascript object>

# Create Episodes from Posts

- Fetch production database dump and apply it locally
- Set site to localhost in wagtailadmin
- Migrate up to add episode migration
- Convert Posts to Episodes
- Recover slugs
- Apply rename attributes and delete old attributes migration
- Dump db and restore to staging

In [3]:
post = Post.objects.first()

<IPython.core.display.Javascript object>

In [5]:
def post_to_episode(post, content_type):
    exclude = {"id", "page_ptr_id", "page_ptr", "translation_key"}
    kwargs = {
        f.name: getattr(post, f.name)
        for f in Post._meta.fields
        if f.name not in exclude
    }
    kwargs["slug"] = f"new_{post.slug}"
    kwargs["content_type"] = content_type
    kwargs["new_keywords"] = post.keywords
    kwargs["new_block"] = post.block
    kwargs["new_podcast_audio"] = post.podcast_audio
    kwargs["new_explicit"] = post.explicit
    return Episode(**kwargs)

<IPython.core.display.Javascript object>

In [6]:
episode_content_type = ContentType.objects.get(app_label="cast", model="episode")
blog = post.blog
for post in Post.objects.all():
    if post.podcast_audio is None:
        continue
    episode = post_to_episode(post, episode_content_type)
    blog.add_child(instance=episode)
    post.delete()

<IPython.core.display.Javascript object>

In [7]:
for episode in Episode.objects.all():
    episode.slug = episode.slug.removeprefix("new_")
    episode.save()

<IPython.core.display.Javascript object>

In [1]:
for episode in Episode.objects.all():
    print(episode.podcast_audio)

7 - Weihnachtsfolge - Episode 3
14 - Machine Learning am Beispiel Bildklassifikation - Episode 8
13 - Machine Learning am Beispiel Textklassifikation - Episode 7
18 - Deployment von Webapplikationen - Episode 12
20 - Python 3.8 - Episode 14
21 - Fragen über Fragen - Episode 15
23 - Python Entwicklungsumgebung 2020 Edition
22 - pp_16
24 - Suchmaschinen
29 - Projektmanagement - "es ist alles nicht so einfach"
26 - Corona aus der Data-Science Perspektive
8 - Python für Einsteiger - Episode 4
5 - Vorstellung und Python-Einführung - Episode 1
15 - Content Management mit Django / Wagtail - Episode 9
12 - Pandas - Episode 6
9 - Datenbanken - Episode 5
16 - Devops - Episode 10
19 - Naive Bayes - Episode 13
17 - Python in der Wissenschaft - Episode 11
28 - Open Source
27 - Javascript Frontends
34 - Python Async Teil 1 - am Beispiel von Django
63 - Ansible
58 - FastAPI
38 - Tests
64 - Crypto / BitCoin / Smart Contracts etc
59 - CSS / Markdown / Microservices
65 - Python 3.11 und Listen
39 - Pyth

In [4]:
#episode = Episode.objects.first()

# Make Sure the Feed is Still Working

In [34]:
import requests

In [35]:
url = "http://localhost:8000/show/feed/podcast/m4a/rss.xml"

In [36]:
r = requests.get(url)

In [37]:
r.status_code

200

In [53]:
#r.content

In [1]:
Post.objects.count()

49

In [43]:
non_episode = Post.objects.first()

In [50]:
isinstance(non_episode.specific, Post)

True

In [39]:
post = Post.objects.last()

In [40]:
post.slug

'jahresruckblick-2022-und-ungelesene-macbooks'

In [48]:
post.specific

<Episode: Jahresrückblick 2022 und ungelesene MacBooks>

In [52]:
isinstance(post.specific, Episode)

True

In [42]:
post.podcast_audio

AttributeError: 'Post' object has no attribute 'podcast_audio'

In [16]:
post.specific.podcast_audio

<Audio: 1 - Jahresrückblick 2022 und eigene Projekte>

In [17]:
post.podcast_audio

AttributeError: 'Post' object has no attribute 'podcast_audio'

In [12]:
Episode.objects.get(slug="design-patterns-draft")

DoesNotExist: Episode matching query does not exist.